In [19]:
import redis 

client = redis.Redis('host.docker.internal', 6379, 0)

client.flushdb()

True

In [20]:
import json
import numpy as np
import threading
import time
from concurrent.futures import ThreadPoolExecutor
from pcomp.kafka_handlers import KafkaProducerHandler, KafkaConsumerHandler, KafkaConsumerHandlerNeuron
from pcomp.activation_functions import ACTIVATIONS, relu, softmax
from pcomp.redis_utils import RedisHandler
from pcomp.parser import parse_layer_coordinator_message, parse_layer_message
from pcomp.avro_utils import avro_serialize, avro_deserialize
from pcomp.neurons_accumulator import NeuronsAccumulator

# Kafka Configuration
KAFKA_BROKER = 'kafka:29092'

In [21]:
class Neuron(threading.Thread):
    def __init__(self, layer_id, neuron_id, weights, bias, activation, is_final_layer):
        threading.Thread.__init__(self)
        self.layer_id = layer_id
        self.layer_id_num = int(self.layer_id.replace("layer_", ""))
        self.neuron_id = neuron_id
        self.weights = np.array(weights)
        self.bias = np.array(bias)
        self.activation = activation
        self.activation_func = ACTIVATIONS.get(activation, relu)
        self.is_final_layer = is_final_layer
        self.redis_handler = RedisHandler('host.docker.internal', 6379, 0, 8)
        self.producer = None
        self.accumulators = {}
        self.neurons_rec = 128 if self.layer_id_num == 1 else 10
        

    def fetch_input_0(self, image_id):
        key = f"streams:{image_id}:initial_data"
        return np.frombuffer(self.redis_handler.get(key), dtype=np.float64)

    def process_0(self, message):
        image_id_str = message
        image_id = image_id_str
        input_data = self.fetch_input_0(image_id)
        z = np.dot(input_data, self.weights) + self.bias
        output = z if self.is_final_layer else self.activation_func(z)
        msg = f"{self.neuron_id}|{image_id}|{format(output, '.17g')}"
        self.producer.send(msg)

    def process(self, message):
        msg = parse_layer_coordinator_message(message)
        image_id = msg.image_id
        neuron_id = msg.neuron_id
        output = msg.output
        try:
            acc = self.accumulators[image_id]
        except KeyError as e:
            acc = self.accumulators[image_id] = NeuronsAccumulator(self.neurons_rec)
        if acc.outputs[neuron_id] is None:
            acc.outputs[neuron_id] = output
            acc.completed += 1
        if acc.completed == self.neurons_rec:
            input_data = np.array(acc.outputs, dtype=np.float64)
            z = np.dot(input_data, self.weights) + self.bias
            output = z if self.is_final_layer else self.activation_func(z)
            msg = f"{self.neuron_id}|{image_id}|{format(output, '.17g')}"
            self.producer.send(msg)
            del self.accumulators[image_id]

    def run(self):
        # Instantiate Kafka consumer and producer inside the thread.
        consumer = KafkaConsumerHandler(f'layer-{self.layer_id_num}', KAFKA_BROKER, group_id=f"{self.neuron_id}_{self.layer_id_num}_group")
        self.producer = KafkaProducerHandler(KAFKA_BROKER, f'layer-{self.layer_id_num + 1}') if not self.is_final_layer else KafkaProducerHandler(KAFKA_BROKER, 'predictions')
        last_msg_time = time.time()
        while True:
            got_message = False
            for message in consumer.consume():
                got_message = True
                last_msg_time = time.time()
                if self.layer_id_num == 0:
                    self.process_0(message)
                else:
                    self.process(message)
            if not got_message and (time.time() - last_msg_time > 10):
                consumer.commit()
                consumer.close()
                self.producer.close()
                break


class OutputLayer(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        self.neurons_rec = 10
        self.redis_handler = RedisHandler('host.docker.internal', 6379, 0, 50)
        self.producer = None
        self.accumulators = {}

    def run(self):
        consumer = KafkaConsumerHandler('predictions', KAFKA_BROKER, group_id=f"predictions_coord_group")
        last_msg_time = time.time()
        while True:
            got_message = False
            for message in consumer.consume():
                got_message = True
                last_msg_time = time.time()
                msg = parse_layer_coordinator_message(message)
                neuron_id = msg.neuron_id
                image_id = msg.image_id
                output = msg.output
                try:
                    acc = self.accumulators[image_id]
                except KeyError:
                    acc = self.accumulators[image_id] = NeuronsAccumulator(self.neurons_rec)
                if acc.outputs[neuron_id] is None:
                    acc.outputs[neuron_id] = output
                    acc.completed += 1
                print(acc.completed)
                if acc.completed == self.neurons_rec:
                    self.aggregate_neuron_outputs(image_id, acc.outputs)
                    del self.accumulators[image_id]
            if not got_message and (time.time() - last_msg_time > 10):
                consumer.commit()
                consumer.close()
                if self.producer:
                    self.producer.close()
                break

    def aggregate_neuron_outputs(self, image_id, local_outputs):
        outputs = np.array(local_outputs, dtype=np.float64)
        prediction = int(np.argmax(outputs))
        self.redis_handler.hset('streams:predictions', image_id, prediction)
        self.redis_handler.delete_streams_keys(image_id)

# Load network and dataset
data = json.load(open("node_based_model.json"))
#df = pd.read_csv('data/mnist.csv').head(10)

neurons = []
output_layer = OutputLayer()

for layer_name, layer_info in data.items():
    neurons += [Neuron(layer_id=layer_name, neuron_id=i, weights=node['weights'], bias=node['biases'], activation=node['activation'], is_final_layer=(layer_name == list(data.keys())[-1])) for i, node in enumerate(layer_info['nodes'])]

# Start all threads
for thread in neurons:
    thread.start()

output_layer.start()

print("Threads started")

Threads started


%4|1744102746.784|OFFSET|rdkafka#consumer-275| [thrd:main]: predictions [5]: offset reset (at offset 1 (leader epoch 0), broker 1) to offset END (leader epoch -1): fetch failed due to requested offset not available on the broker: Broker: Offset out of range
%4|1744102746.784|OFFSET|rdkafka#consumer-275| [thrd:main]: predictions [8]: offset reset (at offset 4 (leader epoch 0), broker 1) to offset END (leader epoch -1): fetch failed due to requested offset not available on the broker: Broker: Offset out of range
%4|1744102748.042|OFFSET|rdkafka#consumer-1671| [thrd:main]: layer-0 [38]: offset reset (at offset 10 (leader epoch 0), broker 1) to offset END (leader epoch -1): fetch failed due to requested offset not available on the broker: Broker: Offset out of range
%4|1744102748.043|OFFSET|rdkafka#consumer-1671| [thrd:main]: layer-0 [9]: offset reset (at offset 95 (leader epoch 0), broker 1) to offset END (leader epoch -1): fetch failed due to requested offset not available on the broker:

1
2
3


%4|1744102800.832|OFFSET|rdkafka#consumer-275| [thrd:main]: predictions [5]: offset reset (at offset 1 (leader epoch 0), broker 1) to offset END (leader epoch -1): fetch failed due to requested offset not available on the broker: Broker: Offset out of range
%4|1744102800.832|OFFSET|rdkafka#consumer-275| [thrd:main]: predictions [8]: offset reset (at offset 4 (leader epoch 0), broker 1) to offset END (leader epoch -1): fetch failed due to requested offset not available on the broker: Broker: Offset out of range
